In [1]:
import numpy as np
import random
import json
import pickle
import nltk
from nltk import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gunaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gunaw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()

In [4]:
intents = json.loads(open('intents.json').read())

In [5]:
words = []
classes = []
docs = []
ignore_punc = ['?', '.', ',', '!', '\'']

for intent in intents['intents']:
    for p in intent['patterns']:
        word_list = nltk.word_tokenize(p)
        words.extend(word_list)
        docs.append((word_list, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_punc]
words = sorted(set(words))
classes = sorted(set(classes))

In [7]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [8]:
train = []
output_empty = [0] * len(classes)

In [9]:
for d in docs:
    bag = []
    word_patterns = d[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    for w in words:
        bag.append(1) if w in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(d[1])] = 1
    train.append([bag, output_row])

In [10]:
random.shuffle(train)
train = np.array(train)

X_train = list(train[:, 0])
y_train = list(train[:, 1])

C:\Users\gunaw\AppData\Local\Temp/ipykernel_9384/2546386037.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train)


In [11]:
model = Sequential()

model.add(Dense(128, input_shape=(len(X_train[0]), ), activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(y_train[0]), activation='softmax'))

sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4608      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 260       
                                                                 
Total params: 13,124
Trainable params: 13,124
Non-trainable params: 0
_________________________________________________________________


C:\Users\gunaw\anaconda3\envs\skripsi_env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
4/4 [==============================] - 0s 2ms/step - loss: 1.4132 - accuracy: 0.1500
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3516 - accuracy: 0.2000
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 1.4006 - accuracy: 0.4000
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3837 - accuracy: 0.4000
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 1.3380 - accuracy: 0.3000
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 1.3170 - accuracy: 0.3000
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2361 - accuracy: 0.5500
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2438 - accuracy: 0.4000
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1871 - accuracy: 0.6000
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 1.1585 - accuracy: 0.5000
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.0373 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1202 - accuracy: 1.0000
Epoch 86/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0496 - accuracy: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0671 - accuracy: 0.9500
Epoch 88/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0577 - accuracy: 1.0000
Epoch 89/200
4/4 [==============================] - 0s 997us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 996us/step - loss: 0.0520 - accuracy: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1502 - accuracy: 0.9000
Epoch 92/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0946 - accuracy: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0525 - accuracy: 1.0000
Epoch 94/200
4/4 [===========

4/4 [==============================] - 0s 997us/step - loss: 0.0667 - accuracy: 0.9500
Epoch 167/200
4/4 [==============================] - 0s 997us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 997us/step - loss: 0.0509 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0180 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 171/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0309 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 997us/step - loss: 0.0061 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0214 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0838 - accuracy: 0.9500
Epoch 176/200
4/

In [13]:
model.save('simple_chatbot.h5')